In [2]:
import re
import string
import pandas as pd
from math import log
import os
import random
import re
import string
from collections import Counter
from sklearn.cluster import KMeans
import pickle

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")
nltk.download("stopwords")


SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

[nltk_data] Error loading stopwords: <urlopen error [Errno 54]
[nltk_data]     Connection reset by peer>


In [2]:
final_counts = []

In [3]:
def clean_text(text):
    text = str(text).lower() 
    text = text.translate (str.maketrans('', '', string.punctuation))
    text = re.sub("\d+", "", text)
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub('[!()-[]{};:\'"\,<>./?@#$%^&*_~–]+', '', text)
    return text

In [20]:
all_non_roman = [] #list to keep all the grievances with more Devnagri words than non-Devnagri


In [9]:
pref = "/Volumes/olaaaa/Thesis_backup/data/"

In [21]:
for file in os.listdir("/Volumes/olaaaa/Thesis_backup/data"):
    #print(file, "here")
    if file.endswith("csv"):
        # print("here")
        df = pd.read_csv(pref+file, header = None, low_memory=False)
        # print(len(df))
        df = df[[0,21]]
        df.columns = ['reg_no', 'subject_content']
        
        df['subject_content_cleaned'] = df['subject_content'].apply(clean_text)
        tmp = [ val for val in df['subject_content_cleaned'].values \
        if len(re.findall("[a-z]+",val)) < 0.5*len(val.split())]
        all_non_roman.extend(tmp) 
        print(file, "Size of updated list", str(len(all_non_roman)))
        # print()
        
        
        

chunk8.csv Size of updated list 226941
chunk9.csv Size of updated list 325551
chunk2.csv Size of updated list 523247
chunk3.csv Size of updated list 592883
chunk1.csv Size of updated list 692915
chunk0.csv Size of updated list 822461
chunk4.csv Size of updated list 825907
chunk5.csv Size of updated list 867773
chunk7.csv Size of updated list 1133099
chunk6.csv Size of updated list 1349852


In [19]:
def row_count(input):
    with open(input) as f:
        for i, l in enumerate(f):
            pass
    return i

Total grievances: 9413682
1. (85.66%) roman(English + Hindi-Latin)(76+24%) 8063829     (calculated previously)
2. (14.34%) non-roman scripts 1255499
   - (13.33%) devnagri script(completely Hindi) 1255499
   - (1.002%) 94353 other scripts
      - (0.057%) bengali 5335 
      - (0.009 %) odia 891 
      - (0.034%) telugu 3223 
      - (0.012%) punjabi 1101
      - (0.095%) kannada 8950 
      - (0.123%) tamil 11580 
      - (0.626%) gujarati 58914 



In [11]:
for file in os.listdir("/Users/joker/Library/CloudStorage/OneDrive-IITKanpur/Thesis/count_languages_devnagri/scripts"):
    print(file, row_count("/Users/joker/Library/CloudStorage/OneDrive-IITKanpur/Thesis/count_languages_devnagri/scripts/" +file), \
        round((row_count("/Users/joker/Library/CloudStorage/OneDrive-IITKanpur/Thesis/count_languages_devnagri/scripts/" +file)*100/9413682),3), "%")

bengali.csv 5335 0.057 %
odia.csv 891 0.009 %
telugu.csv 3223 0.034 %
punjabi.csv 1101 0.012 %
kannada.csv 8950 0.095 %
tamil.csv 11580 0.123 %
gujarati.csv 58914 0.626 %


In [20]:


row_count("/Volumes/olaaaa/Thesis_backup/grievance.csv")

KeyboardInterrupt: 

In [23]:
len(all_non_roman)

1349852

In [34]:
9413681 - 1349852

8063829

In [22]:

with open('all_non_roman.pkl', 'wb') as f:
    pickle.dump(all_non_roman, f)

In [4]:
with open('all_non_roman.pkl', 'rb') as f:
    loaded = pickle.load(f)

In [ ]:
df = pd.DataFrame(loaded)

In [ ]:
df.columns = ['subject_content']

In [ ]:
def find_devnagri(val):
    if len(re.findall("[\u0900-\u097F]+",val)) > len(val.split())/2:
        

In [ ]:
df["isDevnagri"] = df['subject_content'].apply(lambda x: "devnagri" if \
    len(re.findall("[\u0900-\u097F]+",x)) > len(x.split())/2 else "" )

In [8]:
# all_devnagri = [ val for val in df['subject_content'].values \
#         if len(re.findall("[\u0900-\u097F]+",val)) > len(val.split())/2]

In [ ]:
df.isDevnagri.value_counts()

devnagri    1255499
              94353
Name: isDevnagri, dtype: int64

In [ ]:
df_non_roman_non_devnagri = df[df.isDevnagri != "devnagri"]

In [ ]:
with open('df_non_roman_non_devnagri.pkl', 'wb') as f:
    pickle.dump(df_non_roman_non_devnagri, f)

In [3]:
with open('df_non_roman_non_devnagri.pkl', 'rb') as f:
    df_non_roman_non_devnagri = pd.DataFrame(pickle.load(f))

In [14]:
def freq(sent):
    arr = np.zeros(1000)
    for letter in sent:
        if 2300 <= ord(letter) < 3300:
            # print("here")
            arr[ord(letter) - 2300] += 1 
    #arr[::-1].sort()
    
    return arr / np.linalg.norm(arr) #normalization

In [19]:
sent = "আমার নাম জয়ন্ত মণ্ডল আমার স্ত্রীর নাম ডোনা রায় মণ্ডল আমাদের"
sent = "గౌరవనియులైన ప్రధానమంత్రి శ్రీ నరేంద్రమోడీ గారికి మన సరిహద్దుల్లో చైనా దూరాక్రమణకు సరైన రీతి లో జవాబు"
# sent = "નમસ્કાર પ્રધાનમંત્રીજી હું એક મોરબીનો સામાન્ય નાગરિક છુંહાલ મોરબી ના રોડ ની હાલત ખુબ જ ખરાબ છે હું વિનંતી કરું છું કે આ સમસ્યાનો"
sent = "வணக்கத்திற்கும் போற்றுதலுக்கும் உரிய தலைசிறந்த பாரத பிரதமர் அவர்களுக்கு வணக்கம் தங்களின் சிறந்த"
uni = []
for letter in sent:
    uni.append(ord(letter))
print(sorted(uni))


[32, 32, 32, 32, 32, 32, 32, 32, 32, 2949, 2953, 2965, 2965, 2965, 2965, 2965, 2965, 2965, 2965, 2965, 2965, 2965, 2969, 2970, 2970, 2979, 2979, 2980, 2980, 2980, 2980, 2980, 2980, 2980, 2980, 2980, 2984, 2984, 2985, 2986, 2986, 2986, 2990, 2990, 2990, 2990, 2991, 2992, 2992, 2992, 2992, 2992, 2993, 2993, 2993, 2993, 2993, 2994, 2994, 2995, 2995, 2997, 2997, 2997, 3006, 3007, 3007, 3007, 3007, 3007, 3007, 3009, 3009, 3009, 3009, 3009, 3009, 3016, 3019, 3021, 3021, 3021, 3021, 3021, 3021, 3021, 3021, 3021, 3021, 3021, 3021, 3021, 3021, 3021, 3021]


In [5]:
df_non_roman_non_devnagri['histograms'] = df_non_roman_non_devnagri['subject_content'].apply(freq)

In [ ]:
df_non_roman_non_devnagri.isna().sum()

subject_content    0
isDevnagri         0
histograms         0
dtype: int64

In [22]:
df_non_roman_non_devnagri.head()

,subject_content,isDevnagri,histograms
13,respected sirmadam আমার নাম জয়ন্ত মণ্ডল আমার ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
30,மதிப்பிற்குரிய ஐயா வணக்கம் பொருள் எரிவாயு சப்ள...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
40,নমস্কার স্যার আমি আপনার কাছে একটাই অনুরোধ যে আ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
52,ঘার কে লিয়ে এপ্লাই,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
58,স্যার আমি একজন গরীব কৃষক কুলতলী থানার কাওড়াখাল...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [23]:
with open('df_non_roman_non_devnagri_histograms.pkl', 'wb') as f:
    pickle.dump(df_non_roman_non_devnagri, f)

In [3]:
with open('df_non_roman_non_devnagri_histograms.pkl', 'rb') as f:
    df = pickle.load(f)

In [4]:
clusters = 7
def mbkmeans_clusters(X, k):
    km = KMeans(n_clusters=k).fit(X)
    with open("km_non_roman_non_devnagri.pkl", "wb") as f:
        pickle.dump(km, f)
    return km.labels_

In [5]:
df

,subject_content,isDevnagri,histograms
13,respected sirmadam আমার নাম জয়ন্ত মণ্ডল আমার ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
30,மதிப்பிற்குரிய ஐயா வணக்கம் பொருள் எரிவாயு சப்ள...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
40,নমস্কার স্যার আমি আপনার কাছে একটাই অনুরোধ যে আ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
52,ঘার কে লিয়ে এপ্লাই,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
58,স্যার আমি একজন গরীব কৃষক কুলতলী থানার কাওড়াখাল...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
1349791,માનનીય સાહેબ જીતના અભિનંદન પણ દાહોદ ની જીતથી હ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1349793,மேதகு பாரதபிரதமர் அவர்களுக்கு பணிவான முதல் வணக...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1349808,মোৰ শ্ৰদ্ধাৰ মৰমৰ আদৰণীয় সন্মানীয় মাননীয় ভাৰতৰ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1349822,பாரத் மாதாகி ஜெய் பாரத் மாதாகி ஜெய் பாரத் மாதா...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [6]:
df.reset_index(drop=True, inplace=True)

In [7]:
df = df.loc[ df.histograms.map(lambda x: ~np.isnan(x).any()), :]


In [8]:
df['labels'] = mbkmeans_clusters(
    X=list(df['histograms'].values),
    k=clusters,
)

In [10]:
df.labels.value_counts()

1    58914
2    11580
0     8950
6     5335
4     3223
5     1101
3      891
Name: labels, dtype: int64

In [11]:
df

,subject_content,isDevnagri,histograms,labels
0,respected sirmadam আমার নাম জয়ন্ত মণ্ডল আমার ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
1,மதிப்பிற்குரிய ஐயா வணக்கம் பொருள் எரிவாயு சப்ள...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
2,নমস্কার স্যার আমি আপনার কাছে একটাই অনুরোধ যে আ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
3,ঘার কে লিয়ে এপ্লাই,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
4,স্যার আমি একজন গরীব কৃষক কুলতলী থানার কাওড়াখাল...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
...,...,...,...,...
94348,માનનીય સાહેબ જીતના અભિનંદન પણ દાહોદ ની જીતથી હ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
94349,மேதகு பாரதபிரதமர் அவர்களுக்கு பணிவான முதல் வணக...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
94350,মোৰ শ্ৰদ্ধাৰ মৰমৰ আদৰণীয় সন্মানীয় মাননীয় ভাৰতৰ...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
94351,பாரத் மாதாகி ஜெய் பாரத் மாதாகி ஜெய் பாரத் மாதா...,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2


In [11]:
df[df.labels == 2].subject_content.to_csv("/Volumes/olaaaa/Thesis_backup/count_languages_devnagri/scripts/"+str(2)+".csv")

In [12]:
for i in range(clusters):
    print(df[df.labels == i])
    df[df.labels == i].subject_content.to_csv("/Volumes/olaaaa/Thesis_backup/count_languages_devnagri/scripts/"+str(i)+".csv")

                                         subject_content isDevnagri  \
8      ಭಾರತ ಮತ್ತು ಚೀನಾ ನಡುವಿನ ಯುದ್ಧದಲ್ಲಿ ನನಗೆ ಭಾಗವಹಿಸ...              
9      ಭಾರತ ಮತ್ತು ಚೀನಾ ನಡುವಿನ ಯುದ್ಧದಲ್ಲಿ ನನಗೆ ಭಾಗವಹಿಸ...              
23     dear sir i m ganesh kundapura ಸರ್ ಭಾರತ ಚೀನಾದ ಗ...              
73     ಭಾರತದ ಪ್ರಧಾನ ಮಂತ್ರಿಗಳಾದ ಶ್ರೀ ನರೇಂದ್ರ ದಾಮೋದರದಾಸ...              
95     ಸರ್ ನನ್ನದೊಂದು ವಿನಂತಿ ನಾನು sansera engineering ...              
...                                                  ...        ...   
89959  dear sir ಯಾವ ಧರ್ಮವನ್ನು ದ್ವೇಷಿಸಲಿಲ್ಲ ಯಾವ ಜಾತಿಯನ...              
89969  ಬಿಜೆಪಿ ಅಭೂತಪೂವ ಗೆಲುವಿನಲ್ಲಿ ಪ್ರಮುಖ ಪಾತ್ರ ವಹಿಸಿದ...              
89978   ಲೋಕಸಭಾ ಚುಣಾವನೆಯಲ್ಲಿ ಮುನ್ನಡೆ ಸಾಧಿಸಿರುವ ನನ್ನ ಆತ...              
89982  ಪ್ರಧಾನ ಮಂತ್ರಿಗಳು ಶ್ರೀ ಮೋದಿಯವರಿಗೆ ಹೃದಯಪೂರ್ವಕ ಜಯ...              
89983  ಪ್ರಧಾನ ಮಂತ್ರಿ ಗಳಲ್ಲಿ ಮನವಿ ಸಮುದ್ರಕ್ಕೆ ಹತ್ತಿರವಿರ...              

                                              histograms  labels  
8      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       0  
9      [0.0, 

In [13]:
with open("km_non_roman_non_devnagri.pkl", "rb") as f:
        model = pickle.load(f)

In [24]:
model.predict([freq("તમે મહાભારત કે રામાયણ માં શહીદ શબ્દ સાંભળ્યો આ શહીદ શબ્દ ઇસ્લામિક છે શહીદ એનેજ કહેવાય જે ઇસ્લામ માટે જેહાદ કરતા મર્યો હોય પણ અજ્ઞાન વશ આજે આપણે વીર જવાનો ને શહીદ શહીદ કહીએ છીએ જે યોગ્ય નથી આ ઉલટી પ્રથા બદલવી પડશે આ ગ્રુપ થી શરૂઆત કરીએ હવે કોઈ વીર જવાન ને શહિદ થયો એમ નહિ કહે વીરગતિ ને પ્રાપ્ત થયો તેમ જ કહેવાનું જ્યારથી શહીદ શબ્દ નો અર્થ સમજ્યો છું ત્યારથી હું શહીદ શબ્દ નો ઉપયોગ કરવાનું ટાલુ છું"),\
    freq("ಆತ್ಮೀಯ ಪ್ರಧಾನ ಮಂತ್ರಿಯವರಿಗೆ ನಾನೂ ಹೇಳುಹುದೇನೆಂದರೆ ತುಂಬಾ ಜನರಿಗೆ ಗೊತ್ತಿಲ್ಲ ಯಾವುವು ಭಾರತ ದೇಶದ ಉತ್ಪನ್ನಗಳು"),
    freq("உலகின் மிகப்பெரிய அணையின் பெயர் என்ன")])

array([1, 0, 2], dtype=int32)

In [ ]:
model.predict([freq("તમે મહાભારત કે રામાયણ માં શહીદ શબ્દ સાંભળ્યો આ શહીદ શબ્દ ઇસ્લામિક છે શહીદ એનેજ કહેવાય જે ઇસ્લામ માટે જેહાદ કરતા મર્યો હોય પણ અજ્ઞાન વશ આજે આપણે વીર જવાનો ને શહીદ શહીદ કહીએ છીએ જે યોગ્ય નથી આ ઉલટી પ્રથા બદલવી પડશે આ ગ્રુપ થી શરૂઆત કરીએ હવે કોઈ વીર જવાન ને શહિદ થયો એમ નહિ કહે વીરગતિ ને પ્રાપ્ત થયો તેમ જ કહેવાનું જ્યારથી શહીદ શબ્દ નો અર્થ સમજ્યો છું ત્યારથી હું શહીદ શબ્દ નો ઉપયોગ કરવાનું ટાલુ છું"),\
    freq("ಆತ್ಮೀಯ ಪ್ರಧಾನ ಮಂತ್ರಿಯವರಿಗೆ ನಾನೂ ಹೇಳುಹುದೇನೆಂದರೆ ತುಂಬಾ ಜನರಿಗೆ ಗೊತ್ತಿಲ್ಲ ಯಾವುವು ಭಾರತ ದೇಶದ ಉತ್ಪನ್ನಗಳು"),
    freq("உலகின் மிகப்பெரிய அணையின் பெயர் என்ன")])

array([1, 0, 2], dtype=int32)